In [23]:
import geopandas as gpd
import pandas as pd
from fiona.drvsupport import supported_drivers
import pandas as pd
from shapely.geometry import Point

# Enable KML support
supported_drivers['KML'] = 'rw'

# Load KML file
gdf = gpd.read_file("Pasture.kml", driver="KML")

# Display as a well-formatted table
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.max_colwidth", None) # Show full text values
# display(gdf)  # Works in Jupyter Notebook

def decimal_to_dms(value, is_latitude=True):
    """Convert decimal degrees to degrees, minutes, and seconds (DMS) format."""
    degrees = int(value)
    minutes = int((abs(value) - abs(degrees)) * 60)
    seconds = round((abs(value) - abs(degrees) - minutes / 60) * 3600, 1)
    
    direction = ('N' if value >= 0 else 'S') if is_latitude else ('E' if value >= 0 else 'W')
    return f"{abs(degrees)}°{minutes:02d}'{seconds:04.1f}\"{direction}"

In [24]:
# Convert the 'geometry' column (POINT Z) to only X and Y
# gdf[['Longitude', 'Latitude']] = gdf['geometry'].apply(lambda g: pd.Series([g.x, g.y]))
# gdf[['Longitude', 'Latitude']] = gdf['geometry'].apply(lambda g: pd.Series([round(g.x, 5), round(g.y, 5)]))

gdf[['Longitude', 'Latitude']] = gdf['geometry'].apply(
    lambda g: pd.Series([decimal_to_dms(g.y, is_latitude=True), decimal_to_dms(g.x, is_latitude=False)])
)

# Drop the original 'geometry' column
gdf = gdf.drop(columns=['geometry'])

# Display the table
# print(gdf)


C:\ProgramData\anaconda3\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [25]:
# Identify paddocks by detecting new starts (where 'Name' == 1)
gdf['Paddock'] = (gdf['Name'].astype(int) == 1).cumsum()

# Combine Latitude and Longitude into a formatted multi-line string
gdf['Coordinates'] = gdf.apply(lambda row: f"{row['Latitude']}\n{row['Longitude']}", axis=1)

# Drop unnecessary columns
gdf = gdf[['Paddock', 'Name', 'Coordinates']]

# Pivot table to have one row per paddock, with coordinate strings as columns
pivoted_gdf = gdf.pivot(index='Paddock', columns='Name', values='Coordinates')

# Rename columns for clarity
pivoted_gdf.columns = [f'Point_{int(col)}' for col in pivoted_gdf.columns]

# Reset index for a clean DataFrame
pivoted_gdf.reset_index(inplace=True)
transposed_gdf = pivoted_gdf.set_index("Paddock").T


In [26]:
transposed_gdf

Paddock,1,2,3,4,5,6,7
Point_1,"69°30'41.7""E\n54°13'05.8""N","69°30'41.7""E\n54°13'05.8""N","69°31'06.1""E\n54°12'55.7""N","69°31'30.3""E\n54°12'56.9""N","69°31'55.2""E\n54°12'57.0""N","69°31'55.7""E\n54°12'51.7""N","69°31'56.1""E\n54°12'47.0""N"
Point_2,"69°30'29.2""E\n54°13'05.5""N","69°30'47.1""E\n54°13'03.3""N","69°31'10.6""E\n54°12'55.5""N","69°31'54.8""E\n54°12'56.8""N","69°31'55.7""E\n54°12'51.9""N","69°31'56.1""E\n54°12'47.2""N","69°31'56.5""E\n54°12'41.4""N"
Point_3,"69°30'29.6""E\n54°12'59.0""N","69°30'50.4""E\n54°13'01.1""N","69°31'30.1""E\n54°12'56.9""N","69°30'45.8""E\n54°12'43.5""N","69°30'44.6""E\n54°12'42.0""N","69°30'44.8""E\n54°12'42.1""N","69°30'45.1""E\n54°12'42.1""N"
Point_4,"69°30'35.2""E\n54°12'53.6""N","69°30'51.3""E\n54°12'57.7""N","69°30'45.8""E\n54°12'43.5""N","69°30'45.6""E\n54°12'43.2""N","69°30'45.8""E\n54°12'43.5""N","69°30'45.2""E\n54°12'42.7""N","69°30'44.8""E\n54°12'42.3""N"
Point_5,"69°30'35.7""E\n54°12'42.6""N","69°30'55.2""E\n54°12'56.7""N","69°30'44.6""E\n54°12'43.9""N",NaN,NaN,NaN,NaN
Point_6,"69°30'42.0""E\n54°12'42.1""N","69°31'06.1""E\n54°12'55.7""N",NaN,NaN,NaN,NaN,NaN
Point_7,"69°30'43.3""E\n54°12'44.2""N","69°30'44.5""E\n54°12'43.9""N",NaN,NaN,NaN,NaN,NaN
Point_8,NaN,"69°30'43.4""E\n54°12'44.2""N",NaN,NaN,NaN,NaN,NaN
Point_9,NaN,"69°30'42.9""E\n54°13'02.8""N",NaN,NaN,NaN,NaN,NaN


In [27]:
transposed_gdf.to_excel("paddock_coordinates.xlsx", index=False)